In [2]:
def text2paragraphs(filename, min_size=1):
    """ A text contained in the file 'filename' will be read 
    and chopped into paragraphs.
    Paragraphs with a string length less than min_size will be ignored.
    A list of paragraph strings will be returned"""
    
    txt = open(filename).read()
    paragraphs = [para for para in txt.split("\n\n") if len(para) > min_size]
    return paragraphs


In [3]:
# the position of lables is very important
# it corresponds to a novel by that author within "files"
# the position of the author is also relevant, as it will correspond to metrics
# i.e. Samuel Butler's metrics are always returned in position 1
labels = ['Virginia Woolf', 'Samuel Butler', 'Herman Melville', 
          'David Herbert Lawrence', 'Daniel Defoe', 'James Joyce']


# names of books we have to train our machine model
files = ['night_and_day_virginia_woolf.txt', 'the_way_of_all_flash_butler.txt',
         'moby_dick_melville.txt', 'sons_and_lovers_lawrence.txt',
         'robinson_crusoe_defoe.txt', 'james_joyce_ulysses.txt']

# location of our books
path = "books/"


In [4]:
data = []
targets = []
counter = 0

# loop across all files we have downloaded
for fname in files:
    paras = text2paragraphs(path + fname, min_size=150) # return a book with paragraphs over 150 chars in a list
    data.extend(paras)
    targets += [counter] * len(paras)
    counter += 1


In [5]:
# cell is useless, because train_test_split will do the shuffling!

import random

data_targets = list(zip(data, targets))
# create random permutation on list:
data_targets = random.sample(data_targets, len(data_targets))

data, targets = list(zip(*data_targets))


In [6]:
from sklearn.model_selection import train_test_split

res = train_test_split(data, targets, 
                       train_size=0.8,
                       test_size=0.2,
                       random_state=42)
train_data, test_data, train_targets, test_targets = res 


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

vectorizer = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)

vectors = vectorizer.fit_transform(train_data)

# creating a classifier
classifier = MultinomialNB(alpha=.01)
classifier.fit(vectors, train_targets)

vectors_test = vectorizer.transform(test_data)

predictions = classifier.predict(vectors_test)
accuracy_score = metrics.accuracy_score(test_targets, 
                                        predictions)
f1_score = metrics.f1_score(test_targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("F1-score: ", f1_score)


accuracy score:  0.9156232988568318
F1-score:  0.912133740108433


In [8]:
# we want to use paragraphs from this 2nd Virginia Wolf 
paras = text2paragraphs(path + "the_voyage_out_virginia_woolf.txt", min_size=250)

# start on paragraph 100 and go to paragraph 500
first_para, last_para = 100, 500
vectors_test = vectorizer.transform(paras[first_para: last_para]) # pass a list of strings that will be used to make predictions against
#vectors_test = vectorizer.transform(["To be or not to be"])

predictions = classifier.predict(vectors_test) # make our predictions
print(predictions)
targets = [0] * (last_para - first_para)
accuracy_score = metrics.accuracy_score(targets, 
                                        predictions)
precision_score = metrics.precision_score(targets, 
                                          predictions, 
                                          average='macro')

f1_score = metrics.f1_score(targets, 
                            predictions, 
                            average='macro')

print("accuracy score: ", accuracy_score)
print("precision score: ", accuracy_score)
print("F1-score: ", f1_score)


[5 2 0 3 0 5 5 0 2 5 0 0 1 0 0 0 0 0 1 0 1 0 0 5 1 5 0 1 1 0 0 0 5 2 2 3 0
 2 2 3 0 0 0 0 0 3 0 2 0 0 0 4 2 5 2 0 0 0 0 1 0 5 5 0 3 2 0 0 0 0 5 5 5 5
 0 0 0 0 0 2 2 0 0 2 2 0 5 3 2 5 0 0 0 0 0 5 1 0 1 0 0 3 5 5 2 5 5 5 5 0 5
 0 0 0 0 0 0 3 2 0 0 0 2 0 1 2 2 0 5 5 0 3 0 0 0 0 0 5 1 5 0 0 0 0 0 0 0 0
 0 0 5 1 0 3 5 1 1 1 1 0 0 0 0 1 0 5 0 1 0 0 0 5 5 5 5 0 2 0 0 3 0 0 0 0 0
 5 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 5 5 0 0 0 5 5 5 2 0 5 2 0 0 0 0 0 5 0
 0 5 5 0 0 0 0 2 3 0 0 0 0 5 0 0 5 3 5 1 0 5 5 5 0 0 5 0 1 0 0 0 0 0 0 1 5
 1 1 0 5 5 5 5 2 0 0 0 0 5 0 2 2 0 1 0 0 0 0 0 2 3 0 4 0 0 5 0 1 0 0 0 0 1
 1 0 0 5 5 0 5 0 0 0 0 0 5 3 0 0 0 5 3 1 3 0 1 3 0 0 0 0 0 0 3 0 0 0 0 0 5
 3 3 5 0 3 0 0 0 1 0 1 0 0 0 0 2 1 0 0 0 1 1 0 0 0 0 0 3 0 0 0 0 2 2 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 3 3 0 0 0 0 1 5 0 0 0 0 0 0 0 0]
accuracy score:  0.5925
precision score:  0.5925
F1-score:  0.12401883830455258


In [9]:
# perform a probability test
predictions = classifier.predict_proba(vectors_test)
print(predictions)


[[3.44995708e-005 6.02111147e-006 4.11410543e-007 7.01456262e-005
  2.13056748e-015 9.99888922e-001]
 [3.12178665e-001 3.96198947e-002 4.11273006e-001 5.92884327e-011
  1.22236065e-012 2.36928434e-001]
 [9.99443944e-001 1.00109788e-010 5.18250707e-014 1.07662137e-010
  3.20043953e-016 5.56055304e-004]
 ...
 [9.99999930e-001 7.01889760e-008 2.46716997e-040 8.34453065e-026
  1.01002862e-051 1.20093119e-033]
 [9.99999998e-001 1.89924437e-010 3.34447429e-029 7.47722321e-034
  7.50652165e-056 1.41765519e-009]
 [1.00000000e+000 8.79447870e-053 5.44967099e-067 8.64631132e-086
  9.23990586e-112 3.04424642e-048]]


In [10]:
for i in range(0, 10):
    print(predictions[i], paras[i+first_para])


[3.44995708e-05 6.02111147e-06 4.11410543e-07 7.01456262e-05
 2.13056748e-15 9.99888922e-01] "That's the painful thing about pets," said Mr. Dalloway; "they die. The
first sorrow I can remember was for the death of a dormouse. I regret to
say that I sat upon it. Still, that didn't make one any the less sorry.
Here lies the duck that Samuel Johnson sat on, eh? I was big for my
age."
[3.12178665e-01 3.96198947e-02 4.11273006e-01 5.92884327e-11
 1.22236065e-12 2.36928434e-01] "Please tell me--everything." That was what she wanted to say. He had
drawn apart one little chink and showed astonishing treasures. It seemed
to her incredible that a man like that should be willing to talk to her.
He had sisters and pets, and once lived in the country. She stirred her
tea round and round; the bubbles which swam and clustered in the cup
seemed to her like the union of their minds.
[9.99443944e-01 1.00109788e-10 5.18250707e-14 1.07662137e-10
 3.20043953e-16 5.56055304e-04] The talk meanwhile raced pa